<a href="https://colab.research.google.com/github/laucio/colab/blob/main/Te_damos_la_bienvenida_a_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

Una gran Rama de HPC es el procesamiento de lenguaje natural, que es una rama de la inteligencia artificial que ayuda a las computadoras a entender, interpretar y manipular el lenguaje humano.

En el siguiente ejercicio vamos a partir de una cadena de texto, a la que vamos a aplicar una cadena de funciones para normalizarlo.

El procesamiento de lenguaje natural nos puede servir para entender el sentimiento del texto, poder realizar acciones computacionales en base a ello o darle una connotación tanto positiva o negativa a la oración analizada, entre otras aplicaciones.


# Armado Ambiente


In [10]:
!pip install pycuda
!pip install lorem
!pip install contractions


     |████████████████████████████████| 284 kB 5.9 MB/s 
     |████████████████████████████████| 321 kB 42.8 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85452 sha256=7a2f56dd5d7c3f8e3b5c5eacc0dea5a4b7cbcd88403d49a3e7f9ccb26962d173
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [28]:
import lorem
import pycuda.driver as cuda
import pycuda.autoinit
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')

texto = """Nosotros y nuestros socios almacenamos o accedemos a información en un dispositivo, tales como cookies, y procesamos datos personales, tales como identificadores únicos e información estándar enviada por un dispositivo, para anuncios y contenido personalizados, medición de anuncios y del contenido e información sobre el público, así como para desarrollar y mejorar productos.
Con su permiso, nosotros y nuestros socios podemos utilizar datos de localización geográfica precisa e identificación mediante las características de dispositivos. Puede hacer clic para otorgarnos su consentimiento a nosotros y a nuestros socios para que llevemos a cabo el procesamiento previamente descrito. De forma alternativa, puede acceder a información más detallada y cambiar sus preferencias antes de otorgar o negar su consentimiento.
Tenga en cuenta que algún procesamiento de sus datos personales puede no requerir de su consentimiento, pero usted tiene el derecho de rechazar tal procesamiento. Sus preferencias se aplicarán solo a este sitio web. Puede cambiar sus preferencias en cualquier momento entrando de nuevo en este sitio web o visitando nuestra política de privacidad.
En el Banco Central señalaron que hoy la tasa de interés para estas operaciones ronda -promedio- el 43 % anual Para el caso de pasajes y paquetes de turismo interno, los mismos se podrán comercializar a través del programa Ahora 12.

Textualmente, la comunicación A 7407 difundida este jueves dice: “A partir del 26.11.21 las entidades financieras y no financieras emisoras de tarjetas de crédito no deberán financiar en cuotas las compras efectuadas mediante tarjetas de crédito de sus clientes –personas humanas y jurídicas– de pasajes al exterior y demás servicios turísticos en el exterior (tales como alojamiento, alquiler de auto, etc.), ya sea realizadas en forma directa con el prestador del servicio o indirecta, a través de agencia de viajes y/o turismo, plataformas web u otros intermediarios.”

La nueva restricción que anunció el Banco Central refuerza la política de cuidar a como de lugar las cada vez más escasas reservas netas que administra el organismo que preside Miguel Pesce. El temor de las autoridades es que con la apertura de fronteras, las menores restricciones para ingresar a otros países y la llegada de la temporada de vacaciones la demanda de dólares por turismo se incrementará sensiblemente, aumentando la presión sobre las reservas.

Según el último informe del mercado cambiario elaborado por el Banco Central, “las denominadas “personas humanas” compraron de forma neta US$ 326 millones, básicamente para gastos efectuados con tarjetas por consumos con proveedores no residentes (con un neto de US$ 149 millones, mostrando un incremento de 14% con respecto al mes anterior) y para atesoramiento (con un neto de US$ 134 millones en billetes, con un incremento de 8% respecto al mes previo y un descenso de 79% interanual). Asimismo, se realizaron transferencias de fondos hacia cuentas propias en el exterior por US$ 36 millones contra débitos en cuentas locales en moneda extranjera (“Canjes”), con efecto neutro en el resultado del mercado de cambios”.
"""

texto_tokenizado = nltk.word_tokenize(texto)

def borrar_caracteres_no_ascii(palabras):
    "Borro caracteres no ascii"
    nuevas_palabras = []
    for palabra in palabras:
        nueva_palabra = unicodedata.normalize('NFKD', palabra).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        nuevas_palabras.append(nueva_palabra)
    return nuevas_palabras


def to_lowercase(palabras):
    "Convierto todas las palabras a lowercase"
    nuevas_palabras = []
    for palabra in palabras:
        nueva_palabra = palabra.lower()
        nuevas_palabras.append(nueva_palabra)
    return nuevas_palabras

def remover_puntuacion(palabras):
    "Borro signos de puntuacion"
    nuevas_palabras = []
    for palabra in palabras:
        nueva_palabra = re.sub(r'[^\w\s]', '', palabra)
        if nueva_palabra != '':
            nuevas_palabras.append(nueva_palabra)
    return nuevas_palabras

def reemplazo_numeros(palabras):
    "Reemplazo los numeros por su denominacion en texto"
    p = inflect.engine()
    nuevas_palabras = []
    for palabra in palabras:
        if palabra.isdigit():
            nueva_palabra = p.number_to_words(palabra)
            nuevas_palabras.append(nueva_palabra)
        else:
            nuevas_palabras.append(palabra)
    return nuevas_palabras

def borrar_stopwords(palabras):
    """StopWords son palabras que no aplican mucho significado al texto y pueden ser borradas como La El Etc"""
    nuevas_palabras = []
    for palabra in palabras:
        if palabra not in stopwords.words('spanish'):
            nuevas_palabras.append(palabra)
    return nuevas_palabras


def normalizar(palabras):
    palabras = borrar_caracteres_no_ascii(palabras)
    palabras = to_lowercase(palabras)
    palabras = remover_puntuacion(palabras)
    palabras = reemplazo_numeros(palabras)
    palabras = borrar_stopwords(palabras)
    return palabras

palabras_normalizadas = normalizar(texto_tokenizado)

print(palabras_normalizadas)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['socios', 'almacenamos', 'accedemos', 'informacion', 'dispositivo', 'tales', 'cookies', 'procesamos', 'datos', 'personales', 'tales', 'identificadores', 'unicos', 'informacion', 'estandar', 'enviada', 'dispositivo', 'anuncios', 'contenido', 'personalizados', 'medicion', 'anuncios', 'contenido', 'informacion', 'publico', 'asi', 'desarrollar', 'mejorar', 'productos', 'permiso', 'socios', 'podemos', 'utilizar', 'datos', 'localizacion', 'geografica', 'precisa', 'identificacion', 'mediante', 'caracteristicas', 'dispositivos', 'puede', 'hacer', 'clic', 'otorgarnos', 'consentimiento', 'socios', 'llevemos', 'cabo', 'procesamiento', 'previamente', 'descrito', 'forma', 'alternativa', 'puede', 'acceder', 'informacion', 'mas', 'detallada', 'cambiar', 'preferencias', 'oto

#Conclusiones

Con el ejercicio planteado y resuelto, podemos ver lo poderoso que es el lenguaje python y sus infinitas bibliotecas.

El hecho de poder importar funcionalidad complejas y ejecutarlas facilmente, nos permite abstraernos de la complejidad del algoritmo y enfocarnos en el problema general.

Una vez hecha la normalizacion del texto, existen muchas mas bibliotecas python para seguir procesando y darle valor y sentido a la normalizacion del texto.

Algunas de estas bibliotecas son: TextBlob, Standford CoreNLP, Spacy, Textacy, Gensim, pyLDAvis etc...

#Bibliografía

[Que es el Procesamiento Natural? - IDB blog](https://blogs.iadb.org/conocimiento-abierto/es/que-es-el-procesamiento-de-lenguaje-natural-y-como-ponerlo-en-practica-con-recursos-abiertos/)

[Ntlk docs](https://www.nltk.org/)

[Procesamiento lenguaje natural - Universidad Catalunya](http://datascience.recursos.uoc.edu/es/procesamiento-del-lenguaje-natural-nlp/)